In [15]:
# general
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# preprocessing
import re


In [4]:
df = pd.read_csv('/Users/dhewaradya/Documents/code/gcn-for-app-reviews/scraping/sample1k-review_dataset-020225.csv')
df.head()

,Unnamed: 0,reviewId,content,score,at,replyContent,appVersion,application
0,83368,85d3e765-c7e8-4d15-be91-90b124893c85,Menyenangkan🥰,5,2022-06-30 16:01:21,"Hi Mums, Terima kasih telah menggunakan Teman ...",2.9.2,Teman Bumil
1,43673,edabddf1-7ed6-42b2-830e-32160ccf3bff,Syukaa,5,2020-06-07 21:30:30,"Hai, Parent! Terima kasih atas review positifn...",2.6.17,Asian Parent
2,72119,8aab4534-f359-4b19-b4da-a21ee6bc4a4b,Mentok mulu di PIN stiap mau log in ga bisa ms...,1,2024-09-10 10:48:01,"Hallo Mama Rika Komalasari, kami sangat menger...",2.5.5,Hallobumil
3,80274,10f8f8a7-a82d-4586-8d4e-15b3a6c23a67,Aplikasi sangat membantu saya yang lagi hamil ...,5,2023-08-28 21:21:06,"Hi Mums, Terima kasih telah menggunakan Teman ...",2.19.8,Teman Bumil
4,34588,14a13395-262f-494d-bca6-4173508c4511,Solusi ibu hamil,5,2021-02-26 17:38:04,"Hai, Parent! Terima kasih atas review positifn...",2.8.2,Asian Parent


## Filter after 2021

In [5]:
df["at"] = pd.to_datetime(df["at"])
df = df[df["at"].dt.year >= 2021]

In [8]:
df.count()

Unnamed: 0      6066
reviewId        6066
content         6066
score           6066
at              6066
replyContent    5725
appVersion      5320
application     6066
dtype: int64

In [9]:
df = df[['content']]

# Preprocessing

## Drop Duplicates

In [11]:
df = df.drop_duplicates()
df.count()

content    4816
dtype: int64

## Delete Null

In [13]:
df = df[df != "0"]
df.count()

content    4816
dtype: int64

In [23]:
df = df['content'].astype(str)

In [24]:
df.info()

<class 'pandas.core.series.Series'>
Index: 4816 entries, 0 to 9999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
4816 non-null   object
dtypes: object(1)
memory usage: 75.2+ KB


## Case Folding

In [25]:
# function to process and clean text, remove unecessary text

REPLACE_HTML_RE = re.compile('https?://\s+|www\.\s+')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

def clean_text(words):

    words = words.lower() # lowercase text
    words = REPLACE_BY_SPACE_RE.sub(' ', words) # replace REPLACE_BY_SPACE_RE symbols by space in text
    words = BAD_SYMBOLS_RE.sub('', words) # delete symbols which are in BAD_SYMBOLS_RE from text
    words = REPLACE_HTML_RE.sub('', words) # delete html address
    return words

In [26]:
dfp = df.apply(clean_text)

In [28]:
# function to remove emojis

def remove_emoji(words):
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', words)

In [29]:
dfp = dfp.apply(remove_emoji)

In [30]:
dfp.head()

0                                         menyenangkan
2    mentok mulu di pin stiap mau log in ga bisa ms...
3    aplikasi sangat membantu saya yang lagi hamil ...
4                                     solusi ibu hamil
5    aplikasi parenting terlengkap dan terbaik di i...
Name: content, dtype: object

## Normalization

### Fix slang words

In [32]:
alay_dict = pd.read_csv('new_kamusalay.csv', names=['original', 'replacement'], encoding='latin-1')

alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

In [33]:
dfp = dfp.apply(normalize_alay)
dfp.head()

0                                         menyenangkan
2    mentok mulu di pin setiap mau log in tidak bis...
3    aplikasi sangat membantu saya yang lagi hamil ...
4                                     solusi ibu hamil
5    aplikasi parenting terlengkap dan terbaik di i...
Name: content, dtype: object